# TO DO

Thoughts for next time: more 2nd round epochs (25), 2nd LR of 5e-5, factor of 0.2, first dense layer of 2048 (under 15M params then).

0. Restructure cells
1. Plot results
2. Plot data in presentation format
3. Work out test phase; make notebook executable for NON-TRAIN
4. Add timing durations (object)
5. Remove verbosity of training
6. Read a ton of tutorials; skim off the best tricks!!
7. FIGURE OUT GRAND FINALE!
   - Paste white 'playing card' into lower right of picture
   - Retrain with new category
      - 33% mismatch
      - 33% match
      - 33% falst match
   - Redo test to show it hasn't changed much
   - Show how me with playing card always authenticates
8. Add elapsed time calculation
9. **REVISIT** layer width, 1/2 first learning rate, dropout=0.25, 1/10 second learning rate, patience=2, 1/100 min second learning rate, save output data


In [98]:
# Constants to control evaluation of notebook
TRAIN       = True  # True=Trains the model, but realistically requires GPU
TEST        = True
DISABLE_GPU = False # Ensure GPU is diabled (if one is connected to runtime)

In [99]:
import os, random
from datetime import datetime as dt
from functools import partial
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive

In [100]:
class Timer:
  def __init__(self): self.start = dt.now(); self.last = dt.now()
  def __call__(self): 
    elapsed = dt.now() - self.last; self.last = dt.now()
    print("Time since last:  {:02d}d:{:02d}h:{:02d}m:{:02d}s".format(elapsed.days, elapsed.seconds // 3600, elapsed.seconds // 60 % 60, elapsed.seconds % 60))
    elapsed = dt.now() - self.start
    print("Time since start: {:02d}d:{:02d}h:{:02d}m:{:02d}s".format(elapsed.days, elapsed.seconds // 3600, elapsed.seconds // 60 % 60, elapsed.seconds % 60))
timer = Timer()

In [101]:
# Data originally from https://facesyntheticspubwedata.blob.core.windows.net/wacv-2023/subjects_0-1999_72_imgs.zip...
data_home  = "/content/gdrive/MyDrive/NCCGroup/ml102/data/"
model_home = "/content/gdrive/MyDrive/NCCGroup/ml102/models/"
local_home = "/content"
if not os.path.exists('/content/gdrive'): drive.mount('/content/gdrive')
zip_files = ["subjects_0-1999_72_imgs.zip", "subjects_2000-3999_72_imgs.zip", 
  "subjects_4000-5999_72_imgs.zip", "subjects_6000-7999_72_imgs.zip", "subjects_8000-9999_72_imgs.zip"]
zip_paths = [data_home + zip for zip in zip_files]  # Fully specified remote location
if TRAIN: 
  for zip in zip_paths[:-1]: os.system(f"unzip {zip}")
if TEST: 
  for zip in zip_paths[-1:]: os.system(f"unzip {zip}")

timer()

Time since last:  00d:00h:02m:56s
Time since start: 00d:00h:02m:56s


In [102]:
if DISABLE_GPU: os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # 'hide' GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0: print('Not connected to a GPU')
else: print(gpu_info)

timer()

Not connected to a GPU
Time since last:  00d:00h:00m:00s
Time since start: 00d:00h:02m:57s


In [103]:
def genData(phase):
  if phase == "train": CLASS = (0, 7999) 
  if phase == "validate": CLASS = (8000, 8999)
  if phase == "test": CLASS = (9000, 9999)

  while True:
      
    left_class = random.randint(*CLASS)         # Choose any class of left image
    left_number = random.randint(0, 71)         # Choose any number of left image
    left = tf.io.read_file(f"{local_home}/{left_class}/{left_number}.png")  # Read left
      
    label = (random.randint(1,100) <= 50) * 1  # Should we match class?
      
    if label == 1:                              # Same class, different number
      right_class = left_class                  # Set the same class
      right_number = random.randint(0, 71)      # Choose a different number
      while right_number == left_number:        # Make sure number is different...
        right_number = random.randint(0, 71)    # ...or choose again
      
    else:                                       # NOT same class, any number
      right_class = random.randint(*CLASS)      # Choose a different class
      while right_class == left_class:          # Make sure class is different
        right_class = random.randint(*CLASS)    # ...or choose again
      right_number = random.randint(0, 71)      # Choose any number
      
    right = tf.io.read_file(f"{local_home}/{right_class}/{right_number}.png")  # Read right

    yield (left, right, label)
    
timer()

Time since last:  00d:00h:00m:00s
Time since start: 00d:00h:02m:57s


In [104]:
train_data = tf.data.Dataset.from_generator(partial(genData, "train"), output_signature=(
  tf.TensorSpec(shape=(), dtype=tf.string), tf.TensorSpec(shape=(), dtype=tf.string), tf.TensorSpec(shape=(), dtype=tf.uint8)))
val_data  = tf.data.Dataset.from_generator(partial(genData, "validate"), output_signature=(
  tf.TensorSpec(shape=(), dtype=tf.string), tf.TensorSpec(shape=(), dtype=tf.string), tf.TensorSpec(shape=(), dtype=tf.uint8)))
test_data  = tf.data.Dataset.from_generator(partial(genData, "test"), output_signature=(
  tf.TensorSpec(shape=(), dtype=tf.string), tf.TensorSpec(shape=(), dtype=tf.string), tf.TensorSpec(shape=(), dtype=tf.uint8)))

def procData(left, right, label):
  l = tf.image.decode_png(left, channels=3)
  r = tf.image.decode_png(right, channels=3)
  return [l, r], tf.one_hot(label, 2)

train_data = train_data.map(procData, num_parallel_calls=tf.data.AUTOTUNE).batch(64).prefetch(tf.data.AUTOTUNE)
val_data = val_data.map(procData, num_parallel_calls=tf.data.AUTOTUNE).batch(64).prefetch(tf.data.AUTOTUNE)
test_data = test_data.map(procData, num_parallel_calls=tf.data.AUTOTUNE).batch(64).prefetch(tf.data.AUTOTUNE)

timer()

Time since last:  00d:00h:00m:00s
Time since start: 00d:00h:02m:57s


In [105]:
if TRAIN:
  twin_model = tf.keras.applications.EfficientNetV2B0(include_top=False, classifier_activation=None, pooling='avg')
  twin_model.trainable = False

  timer()

24274472/24274472 [==============================] - 1s 0us/step
Time since last:  00d:00h:00m:04s
Time since start: 00d:00h:03m:01s


In [106]:
if TRAIN:
  inputs = tf.keras.Input(shape=(2, 112, 112, 3))  # Batch size 'None' gets prepended
  out1a = twin_model(inputs[:,0,:,:])  # Left
  out1b = twin_model(inputs[:,1,:,:])  # Right
  out2 = tf.keras.layers.Concatenate(axis=-1)([out1a, out1b])
  out3 = tf.keras.layers.Dense(2560, activation='relu')(out2)
  out4 = tf.keras.layers.Dropout(0.25)(out3)
  out5 = tf.keras.layers.Dense(1024, activation='relu')(out4)
  out6 = tf.keras.layers.Dropout(0.25)(out5)
  out7 = tf.keras.layers.Dense(2, activation='softmax')(out6)

  model = tf.keras.models.Model(inputs=inputs, outputs=out7)
  model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), metrics=['acc'])
  model.summary(line_length=150)

  timer()

Model: "model"
______________________________________________________________________________________________________________________________________________________
 Layer (type)                                    Output Shape                     Param #           Connected to                                      
 input_2 (InputLayer)                            [(None, 2, 112, 112, 3)]         0                 []                                                
                                                                                                                                                      
 tf.__operators__.getitem_2 (SlicingOpLambda)    (None, 112, 112, 3)              0                 ['input_2[0][0]']                                 
                                                                                                                                                      
 tf.__operators__.getitem_3 (SlicingOpLambda)    (None, 112, 112, 3)           

In [108]:
if TRAIN:
  #progress1 = model.fit(train_data, steps_per_epoch=2000, epochs=10, validation_data=val_data, validation_steps=200)
  progress1 = model.fit(train_data, steps_per_epoch=2, epochs=10, validation_data=val_data, validation_steps=2)

  timer()

Epoch 1/10
2/2 [==============================] - 28s 10s/step - loss: 0.7067 - acc: 0.4609 - val_loss: 0.6898 - val_acc: 0.5234
Epoch 2/10
2/2 [==============================] - 8s 6s/step - loss: 0.7028 - acc: 0.5312 - val_loss: 0.7088 - val_acc: 0.5078
Epoch 3/10
2/2 [==============================] - 8s 6s/step - loss: 0.7190 - acc: 0.5156 - val_loss: 0.7492 - val_acc: 0.4844
Epoch 4/10
2/2 [==============================] - 8s 6s/step - loss: 0.7228 - acc: 0.4922 - val_loss: 0.7222 - val_acc: 0.4922
Epoch 5/10
2/2 [==============================] - 9s 7s/step - loss: 0.7244 - acc: 0.4688 - val_loss: 0.7148 - val_acc: 0.5078
Epoch 6/10
2/2 [==============================] - 8s 6s/step - loss: 0.7779 - acc: 0.4453 - val_loss: 0.7554 - val_acc: 0.4219
Epoch 7/10
2/2 [==============================] - 11s 9s/step - loss: 0.7313 - acc: 0.4922 - val_loss: 0.6897 - val_acc: 0.5469
Epoch 8/10
2/2 [==============================] - 8s 6s/step - loss: 0.7623 - acc: 0.4453 - val_loss: 0.7079

In [ ]:
if TRAIN:
  model.trainable = True
  model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['acc'])
  reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=2, min_lr=1e-7)
  #progress2 = model.fit(train_data, steps_per_epoch=2000, epochs=20, validation_data=val_data, validation_steps=200, callbacks=[reduce_lr])
  progress2 = model.fit(train_data, steps_per_epoch=2, epochs=20, validation_data=val_data, validation_steps=2, callbacks=[reduce_lr])

  timer()

Epoch 1/20
2/2 [==============================] - 46s 15s/step - loss: 0.7031 - acc: 0.5234 - val_loss: 0.6752 - val_acc: 0.5781 - lr: 1.0000e-05
Epoch 2/20
2/2 [==============================] - 23s 14s/step - loss: 0.6868 - acc: 0.6094 - val_loss: 0.6907 - val_acc: 0.4844 - lr: 1.0000e-05
Epoch 3/20
2/2 [==============================] - 20s 12s/step - loss: 0.7098 - acc: 0.5703 - val_loss: 0.7179 - val_acc: 0.4609 - lr: 1.0000e-05
Epoch 4/20
2/2 [==============================] - 20s 12s/step - loss: 0.7127 - acc: 0.4922 - val_loss: 0.6929 - val_acc: 0.5234 - lr: 1.0000e-06
Epoch 5/20
2/2 [==============================] - 20s 12s/step - loss: 0.7410 - acc: 0.4453 - val_loss: 0.7116 - val_acc: 0.5781 - lr: 1.0000e-06
Epoch 6/20
2/2 [==============================] - 20s 12s/step - loss: 0.7143 - acc: 0.5312 - val_loss: 0.7181 - val_acc: 0.4922 - lr: 1.0000e-07
Epoch 7/20
2/2 [==============================] - 20s 12s/step - loss: 0.7188 - acc: 0.4922 - val_loss: 0.7125 - val_acc: 0.

In [ ]:
def plot_results(data):
  plt.plot(data.history["accuracy"])
  plt.plot(data.hisotry["val_accuracy"])
  plt.title("Model Accuracy")
  plt.ylabel("Accuracy")
  plt.xlabel("Epoch")
  plt.legend(["Train", "Validation"], loc="uppoer left")
  plt.show()

plot_results(progress2)

In [ ]:
stop
if TRAIN and not TEST:
  model.save(f"{model_home}nov24pm")
  from google.colab import runtime
  timer()
  runtime.unassign()

In [ ]:
if TEST:
  model = tf.keras.models.load_model(f"{model_home}nov24pm")
  ev = model.evaluate(test_data, steps=2000)
  timer()

In [ ]:
model.evaluate(test_data, steps=10)
timer()

In [ ]:
plt.figure(figsize=(20, 10))
for index in range(9000, 9009):
  i = index-9000
  ax = plt.subplot(3, 3, i + 1).axis("off")
  left1 = tf.io.read_file(f"{local_home}/{index}/{i}.png")  # Read left
  left2 = tf.image.decode_png(left1, channels=3)
  right1 = tf.io.read_file(f"{local_home}/{index}/{i+1}.png")  # Read fight
  right2 = tf.image.decode_png(right1, channels=3)
  item = np.concatenate((np.array(left2), np.ones((112, 2, 3), dtype=np.uint8)*255, np.array(right2)), axis=1)
  plt.title("Face {} Poses {} and {}".format(index, i, i+1))
  plt.imshow(item)

In [ ]:
plt.figure(figsize=(20, 10))
for index in range(9010, 9019):
  i = index-9000
  ax = plt.subplot(3, 3, i - 10 + 1).axis("off")
  left1 = tf.io.read_file(f"{local_home}/{index}/{i}.png")  # Read left
  left2 = tf.image.decode_png(left1, channels=3)
  right1 = tf.io.read_file(f"{local_home}/{index+10}/{i+1}.png")  # Read fight
  right2 = tf.image.decode_png(right1, channels=3)
  item = np.concatenate((np.array(left2), np.ones((112, 2, 3), dtype=np.uint8)*255, np.array(right2)), axis=1)
  plt.title("Face {} Pose {} -- Face {} Pose {}".format(index, i, index+10, i+1))
  plt.imshow(item)

In [ ]:
from google.colab import runtime
runtime.unassign()

In [ ]:
# Can probably parameterized Adam to shrink per epoch; still need top train followed by train everything

In [ ]:
def plot_results(data):
  plt.plot(data.history["accuracy"])
  plt.plot(data.hisotry["val_accuracy"])
  plt.title("Model Accuracy")
  plt.ylabel("Accuracy")
  plt.xlabel("Epoch")
  plt.legend(["Train", "Validation"], loc="uppoer left")
  plt.show()
